In [1]:
using ControlSystems
using DSP
using Plots
gr()
ENV["PLOTS_TEST"] = "true"

"true"

In [2]:
mutable struct PhasorPedal
    N::Int
    fc::Float64
    k::Float64
end

PhasorPedal() = PhasorPedal(2, 440., 0.5)

function bodedata(self::PhasorPedal, f::Vector{Float64})
    ω = 2π*f
    ωc = 2π*self.fc
    A = tf([1., -ωc], [1., ωc])
    G = tf(1. - self.k) + tf(self.k)*A^(self.N)
    mag, phase, _ = bode(G, ω)
    
    return mag[:], unwrap!(phase[:]/180*π, range=π)
end

bodedata (generic function with 1 method)

In [3]:
function main()
    N = 2:2:10
    fc = collect(100.:10.:1000.)
    fc = append!(fc, reverse(fc))
    f = 20000*linspace(0, 1., 750).^2 + 1.
    phasor = PhasorPedal()
    
    anim = @animate for n=N, cutoff=fc
        phasor.N = n
        phasor.fc = cutoff
        mag, phase = bodedata(phasor, f)
        magplot = plot(f, mag, ylim=(0, 1), xscale=:log10, title="N = $n", ylab="Amplitude", legend=false)
        plot!(magplot, [cutoff, cutoff], [0, 1], line=:dot)
        phaseplot = plot(f, phase, ylim=(-5π, 0), xscale=:log10, xlab="Frequency [Hz]", ylab="Phase [rad] ", legend=false)
        plot!(phaseplot, [cutoff, cutoff], [-5π, 0], line=:dot)
        plot(magplot, phaseplot, layout=@layout([a; b]))
    end
    
    gif(anim, "Resource\\Phasor.gif", fps = 15)
end

main (generic function with 1 method)

In [4]:
main()

INFO: Saved animation to C:\Users\Aogiri\Documents\GitHub\JuliaSignalProcessingSamples\Resource\Phasor.gif


Plots.AnimatedGif("C:\\Users\\Aogiri\\Documents\\GitHub\\JuliaSignalProcessingSamples\\Resource\\Phasor.gif")